In [52]:
import numpy as np
import pandas as pd
import xarray as xr
from glob import glob
import pymongo
import pdb
from datetime import datetime, timedelta
from dateutil.relativedelta import *

In [63]:
def create_collection(dbName='trpp', collectionName='trppGrids'):
    dbUrl = 'mongodb://localhost:27017/'
    client = pymongo.MongoClient(dbUrl)
    db = client[dbName]
    coll = db[collectionName]
    coll = init_profiles_collection(coll)
    return coll

def init_profiles_collection(coll):
    try:
        coll.create_index([('time', pymongo.DESCENDING)])
        coll.create_index([('data.nlat', pymongo.DESCENDING)])
        coll.create_index([('data.nlon', pymongo.ASCENDING)])

    except:
        logging.warning('not able to get collections or set indexes')
    return coll

coll = create_collection()

In [57]:
#precFileNames= glob('/storage/TRMM/precip/**/*.nc4')
#filename = precFileNames[0]
#prec = xr.open_dataset(filename, decode_cf=True)
precFileNames = '/storage/TRMM/precip/**/*.nc4'
prec = xr.open_mfdataset(precFileNames)

In [58]:
prec.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:        (date: 35063, nlat: 400, nlon: 1440)
Coordinates:
  * nlon           (nlon) float32 -179.875 -179.625 -179.375 ... 179.625 179.875
  * nlat           (nlat) float32 -49.875 -49.625 -49.375 ... 49.625 49.875
  * date           (date) datetime64[ns] 2004-01-01 ... 2015-12-31T21:00:00
Data variables:
    precipitation  (date, nlon, nlat) float32 dask.array<shape=(35063, 1440, 400), chunksize=(1, 1440, 400)>>

In [59]:
prec.dims

Frozen(SortedKeysDict({'nlon': 1440, 'nlat': 400, 'date': 35063}))

In [68]:
def make_docs(dates):
    docs = []
    for date in dates:
        doc = {}
        doc['_id'] = str(date)
        trg = prec.sel(date=date)
        df = trg.to_dataframe()
        df.reset_index(inplace=True)
        df = df[['nlon', 'nlat', 'precipitation']]
        df = df[df['precipitation'] != 0]
        df.astype(np.float32, inplace=True)
        dataDict = df.to_dict(orient='records')
        doc['data'] = dataDict
        doc['time'] = pd.to_datetime(date)
        docs.append(doc)
    return docs
dates = prec.date.data[0:10]
docs = make_docs(dates)

In [69]:
coll.insert_many(docs)

In [49]:
ts = (date - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')

datetime.utcfromtimestamp(ts)

/home/tyler/anaconda3/envs/AR2/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  """Entry point for launching an IPython kernel.


datetime.datetime(2004, 1, 19, 0, 0)

In [65]:
.008 * 35063 / 10

28.050400000000003